In [1]:
import math
from datetime import datetime
from calendar import monthrange
from typing import Callable, Sequence, Union, Tuple
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from dateutil.relativedelta import relativedelta

WatchDate = Union[str, datetime]
FOMCDates = Sequence[Union[str, datetime]]
OHLCLoader = Callable[[str], pd.DataFrame]

In [2]:
def _normalize_watch_date(watch_date: WatchDate) -> datetime:
    if isinstance(watch_date, str):
        return datetime.strptime(watch_date, "%Y-%m-%d")
    elif isinstance(watch_date, datetime):
        return watch_date
    else:
        raise ValueError("watch_date must be 'YYYY-MM-DD' string or datetime")


def _normalize_fomc_dates(fomc_dates: FOMCDates) -> list[datetime]:
    if all(isinstance(d, datetime) for d in fomc_dates):
        res = list(fomc_dates)
    elif all(isinstance(d, str) for d in fomc_dates):
        res = [datetime.strptime(d, "%Y-%m-%d") for d in fomc_dates]
    else:
        raise ValueError(
            "fomc_dates must be list of datetime or list of 'YYYY-MM-DD' strings"
        )
    return sorted(res)


def _starting_no_fomc_month(
    watch_date: datetime, fomc_dates: list[datetime]
) -> Tuple[int, int]:
    """
    원래 FOMC.starting_no_fomc_month 와 동일한 로직.
    watch_date 이전 구간에서 'No FOMC'인 첫 달을 찾는다.
    """
    fomc_ym = [
        d.strftime("%Y-%m")
        for d in fomc_dates
        if d.strftime("%Y-%m") <= watch_date.strftime("%Y-%m")
    ]
    if not fomc_ym:
        raise ValueError("fomc_dates가 지나치게 과거/미래만 포함하는 것 같습니다.")

    target = watch_date
    while target.strftime("%Y-%m") >= fomc_ym[0]:
        if target.strftime("%Y-%m") not in fomc_ym:
            return target.year, target.month
        target = target - relativedelta(months=1)

    raise ValueError(
        "Starting No-FOMC month not found. FOMC 일정 리스트를 확인해 주세요."
    )


def _ending_no_fomc_month(
    watch_date: datetime, fomc_dates: list[datetime], num_upcoming: int
) -> Tuple[int, int]:
    """
    원래 FOMC.ending_no_fomc_month 로직.
    watch_date 이후 num_upcoming 개의 회의를 포함한 뒤,
    그 다음 'No FOMC' 달을 찾는다.
    """
    fomc_ym_fwd = [
        d.strftime("%Y-%m") for d in fomc_dates if d >= watch_date
    ]
    if not fomc_ym_fwd:
        raise ValueError("watch_date 이후 FOMC 일정이 없습니다.")

    target = watch_date
    fomc_counter = 0
    ending_no_fomc = None

    while target.strftime("%Y-%m") <= fomc_ym_fwd[-1]:
        ym = target.strftime("%Y-%m")
        if ym in fomc_ym_fwd:
            fomc_counter += 1
        else:
            ending_no_fomc = target
            if fomc_counter >= num_upcoming:
                break
        target = target + relativedelta(months=1)

    if fomc_counter < num_upcoming:
        raise ValueError(
            f"num_upcoming={num_upcoming} 회의를 충족하지 못했습니다. fomc_dates 범위를 늘려야 합니다."
        )
    if ending_no_fomc is None:
        raise ValueError("Ending No-FOMC month not found. 일정 리스트를 확인해 주세요.")

    return ending_no_fomc.year, ending_no_fomc.month


def _generate_month_list(
    watch_date: datetime, fomc_dates: list[datetime], num_upcoming: int
) -> list[str]:
    y0, m0 = _starting_no_fomc_month(watch_date, fomc_dates)
    y1, m1 = _ending_no_fomc_month(watch_date, fomc_dates, num_upcoming)

    month_index = pd.date_range(
        start=f"{y0}-{m0:02d}", end=f"{y1}-{m1:02d}", freq="MS"
    )
    return [d.strftime("%Y-%m") for d in month_index]


def _generate_contract_list(month_list: list[str]) -> list[str]:
    """
    CME ZQ 월물 코드 생성 (FOMC.generate_contract_list 와 동일)
    """
    cme_month_codes = {
        1: "F",
        2: "G",
        3: "H",
        4: "J",
        5: "K",
        6: "M",
        7: "N",
        8: "Q",
        9: "U",
        10: "V",
        11: "X",
        12: "Z",
    }
    ym = [s.split("-") for s in month_list]
    return [f"ZQ{cme_month_codes[int(m)]}{y}" for (y, m) in ym]


def _generate_meeting_list(
    month_list: list[str], fomc_dates: list[datetime]
) -> list[str]:
    """
    각 month(YYYY-MM)에 해당 월에 FOMC가 있으면 그 날짜(YYYY-MM-DD), 없으면 'No FOMC'
    """
    res = []
    for ym in month_list:
        yy, mm = ym.split("-")
        yy_i, mm_i = int(yy), int(mm)
        matches = [
            d.strftime("%Y-%m-%d")
            for d in fomc_dates
            if d.year == yy_i and d.month == mm_i
        ]
        res.append(matches[0] if matches else "No FOMC")
    return res


def _generate_order_list(
    watch_date: datetime, month_list: list[str], meeting_list: list[str]
) -> list[int]:
    """
    원래 FOMC.generate_order_list 와 동일한 로직 구현.
    month_list / meeting_list 기준으로,
    watch_date를 기준으로 과거/미래 회의에 음수/양수 번호를 달고,
    회의가 없는 달은 0.
    """
    calc_yr, calc_mn = watch_date.year, watch_date.month
    calc_ym = f"{calc_yr}-{calc_mn:02d}"

    try:
        idx = next(i for i, m in enumerate(month_list) if m == calc_ym)
    except StopIteration:
        raise ValueError("watch_date가 month_list 범위 밖입니다.")

    # watch_date가 회의 있는 달인 경우, 이미 회의가 끝난 상태면 과거로 취급
    if meeting_list[idx] == "No FOMC" or (
        meeting_list[idx] != "No FOMC"
        and datetime.strptime(meeting_list[idx], "%Y-%m-%d") <= watch_date
    ):
        bwd = meeting_list[: idx + 1]
        bwd.reverse()
        fwd = meeting_list[idx + 1 :]
    else:
        bwd = meeting_list[:idx]
        bwd.reverse()
        fwd = meeting_list[idx:]

    # forward: upcoming 회의
    fomc_order_fwd = []
    cnt = 1
    for date_str in fwd:
        if date_str == "No FOMC":
            fomc_order_fwd.append(0)
        else:
            fomc_order_fwd.append(cnt)
            cnt += 1

    # backward: past 회의
    fomc_order_bwd = []
    cnt = -1
    for date_str in bwd:
        if date_str == "No FOMC":
            fomc_order_bwd.append(0)
        else:
            fomc_order_bwd.append(cnt)
            cnt -= 1
    fomc_order_bwd.reverse()

    return fomc_order_bwd + fomc_order_fwd


def build_fomc_calendar_summary(
    watch_date: WatchDate,
    fomc_dates: FOMCDates,
    num_upcoming: int,
) -> pd.DataFrame:
    """
    기존 FOMC 클래스의 핵심 정보를 하나의 DataFrame으로 반환.
    index: YYYY-MM
    columns: ['Contract', 'Meeting', 'Order']
    """

    wd = _normalize_watch_date(watch_date)
    fd = _normalize_fomc_dates(fomc_dates)

    month_list = _generate_month_list(wd, fd, num_upcoming)
    contract_list = _generate_contract_list(month_list)
    meeting_list = _generate_meeting_list(month_list, fd)
    order_list = _generate_order_list(wd, month_list, meeting_list)

    summary = pd.DataFrame(
        {
            "Contract": contract_list,
            "Meeting": meeting_list,
            "Order": order_list,
        },
        index=month_list,
    )
    summary.index.name = "YYYY-MM"

    return summary

In [3]:
def _normalize_ohlc(ohlc_df: pd.DataFrame, symbol: str, loader_name: str) -> pd.DataFrame:
    """
    기존 FedWatch.get_fff_history 와 동일한 검증 + Date index 강제.
    """
    if not isinstance(ohlc_df, pd.DataFrame):
        raise ValueError(
            f"'{loader_name}' did not return a pandas DataFrame for {symbol}."
        )
    if "Close" not in ohlc_df.columns:
        raise ValueError(
            f"'{loader_name}' did not return a DataFrame with 'Close' column for {symbol}."
        )

    # Date 가 index 또는 column에 있어야 함
    if "Date" not in ohlc_df.columns and ohlc_df.index.name != "Date":
        raise ValueError(
            f"'{loader_name}' did not return a DataFrame with 'Date' as index or column for {symbol}."
        )

    # index가 Date인 경우
    if ohlc_df.index.name == "Date":
        if pd.to_datetime(ohlc_df.index, errors="coerce").notna().all():
            ohlc_df.index = pd.to_datetime(ohlc_df.index, format="%Y-%m-%d")
        else:
            raise ValueError(
                f"'{loader_name}' returned non-convertible Date index for {symbol}."
            )

    # 그렇지 않으면 column 'Date'를 index로
    if ohlc_df.index.name != "Date":
        if "Date" in ohlc_df.columns and pd.to_datetime(
            ohlc_df["Date"], errors="coerce"
        ).notna().all():
            ohlc_df["Date"] = pd.to_datetime(ohlc_df["Date"], format="%Y-%m-%d")
            ohlc_df.set_index("Date", inplace=True)
        else:
            raise ValueError(
                f"'{loader_name}' returned non-convertible 'Date' column for {symbol}."
            )

    return ohlc_df


def _add_price_data(
    summary: pd.DataFrame,
    watch_date: datetime,
    ohlc_loader: OHLCLoader,
    loader_name: str,
) -> pd.DataFrame:
    """
    기존 FedWatch.add_price_data 와 동일.
    summary 에 Pstart, Pavg, Pend 열을 추가해서 반환.
    """
    p_start, p_avg, p_end = [], [], []

    watch_month = watch_date.strftime("%Y-%m")
    watch_date_str = watch_date.strftime("%Y-%m-%d")

    for ym, row in summary.iterrows():
        contract_symbol = row["Contract"]
        contract_month = ym
        month_type = row["Meeting"]

        ohlc = _normalize_ohlc(
            ohlc_loader(contract_symbol), contract_symbol, loader_name
        )

        # 미만기 계약
        if contract_month >= watch_month:
            p_avg_i = ohlc[ohlc.index <= watch_date_str].iloc[-1]["Close"]
            p_avg.append(p_avg_i)
        else:
            # 만기 지난 계약 → 그 달 마지막 거래일 기준
            yyyy_mm = datetime.strptime(contract_month, "%Y-%m")
            last_day = monthrange(yyyy_mm.year, yyyy_mm.month)[1]
            last_day_str = datetime(
                yyyy_mm.year, yyyy_mm.month, last_day
            ).strftime("%Y-%m-%d")
            p_avg_i = ohlc[ohlc.index <= last_day_str].iloc[-1]["Close"]
            p_avg.append(p_avg_i)

        if month_type == "No FOMC":
            p_start.append(p_avg_i)
            p_end.append(p_avg_i)
        else:
            p_start.append(0.0)
            p_end.append(0.0)

    summary = summary.copy()
    summary["Pstart"] = p_start
    summary["Pavg"] = p_avg
    summary["Pend"] = p_end
    return summary


def _fill_price_data(
    summary: pd.DataFrame,
) -> pd.DataFrame:
    """
    기존 FedWatch.fill_price_data 로직.
    FOMC 달의 Pstart/Pend 0 값을 앞/뒤 달과 회의 날짜로 보간.
    """
    p_start = summary["Pstart"].to_numpy().astype(float)
    p_avg = summary["Pavg"].to_numpy().astype(float)
    p_end = summary["Pend"].to_numpy().astype(float)

    # forward
    for i in range(1, len(p_avg) - 1):
        if p_start[i] == 0.0 and p_end[i - 1] != 0.0:
            p_start[i] = p_end[i - 1]
        if p_end[i] == 0.0 and p_start[i + 1] != 0.0:
            p_end[i] = p_start[i + 1]

    # backward
    month_index = summary.index.to_list()
    meeting_list = summary["Meeting"].to_list()

    for i in range(len(p_avg) - 2, 0, -1):
        if p_end[i] == 0.0:
            p_end[i] = p_start[i + 1]

        if p_start[i] == 0.0:
            meeting_date = datetime.strptime(meeting_list[i], "%Y-%m-%d")
            days_no = monthrange(meeting_date.year, meeting_date.month)[1]
            m = days_no - meeting_date.day + 1
            n = days_no - m
            p_start[i] = (p_avg[i] - m / (m + n) * p_end[i]) / (n / (m + n))

    summary = summary.copy()
    summary["Pstart"] = p_start
    summary["Pavg"] = p_avg
    summary["Pend"] = p_end
    return summary


def _generate_binary_hike_info(
    summary: pd.DataFrame, num_upcoming: int
) -> pd.DataFrame:
    """
    기존 FedWatch.generate_binary_hike_info 와 동일.
    각 FOMC 회의별 H0/H1(두 개의 25bp 시나리오)와 P0/P1(각 시나리오의 확률)을 계산.
    """
    df = summary.copy()
    df = df[(df["Order"] > 0) & (df["Order"] <= num_upcoming)]

    df["Change"] = ((100 - df["Pend"]) - (100 - df["Pstart"])) / 25 * 100
    df["H0"] = df["Change"].apply(lambda x: int(math.trunc(x) * 25))
    df["H1"] = df["Change"].apply(
        lambda x: int(math.trunc(x) * 25 + 25 * np.sign(x))
    )

    df["P0"] = df["Change"].apply(
        lambda x: 1 - (abs(x) - math.trunc(abs(x)))
    )
    df["P1"] = df["Change"].apply(
        lambda x: (abs(x) - math.trunc(abs(x)))
    )
    return df


def _calc_cum_info(
    lead_size: np.ndarray,
    lag_size: np.ndarray,
    lead_prob: np.ndarray,
    lag_prob: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    두 회의(lead/lag)를 합성하여 누적 hike size 및 probability 계산
    (기존 calc_cum_info)
    """
    size_list = lead_size[:, np.newaxis] + lag_size
    prob_list = lead_prob[:, np.newaxis] * lag_prob

    size_flat = size_list.flatten()
    prob_flat = prob_list.flatten()
    unique_size, idx = np.unique(size_flat, return_inverse=True)
    unique_prob = np.bincount(idx, weights=prob_flat)
    return unique_size, unique_prob


def compute_fedwatch_prob_table(
    watch_date: WatchDate,
    fomc_dates: FOMCDates,
    num_upcoming: int,
    ohlc_loader: OHLCLoader,
    rate_cols: bool = True,
    watch_rate_range: Tuple[float, float] | None = None,
) -> pd.DataFrame:
    """
    FOMC/FedWatch 클래스를 쓰지 않고,
    같은 계산 구조로 CME FedWatch 스타일 확률 테이블을 만드는 함수.

    Parameters
    ----------
    watch_date : str or datetime
    fomc_dates : list[str or datetime]
        전체 FOMC 일정 (과거 + 미래).
    num_upcoming : int
        watch_date 이후 몇 회의까지 포함할지.
    ohlc_loader : callable
        symbol(str) -> OHLC DataFrame('Date' index, 'Close' 컬럼 포함)을 반환하는 함수.
        (tvDatafeed, yfinance 등 자유롭게 사용)
    rate_cols : bool
        True면 컬럼 이름을 실제 타깃금리 구간("5.25-5.50")으로 변환.
        False면 bp change(예: -25, 0, 25) 그대로 유지.
    watch_rate_range : (float, float) or None
        (하한, 상한) 현재 타깃금리. None이면 FRED(DFEDTARL/DFEDTARU)에서 조회.

    Returns
    -------
    fedwatch_df : pd.DataFrame
        index: MultiIndex(WatchDate, FOMCDate)
        columns: 개별 금리구간(또는 bp change)별 확률
    """

    wd = _normalize_watch_date(watch_date)
    fd = _normalize_fomc_dates(fomc_dates)

    # 1) FOMC 캘린더 요약
    summary = build_fomc_calendar_summary(wd, fd, num_upcoming)

    # 2) 월별 선물가격 (평균/시작/끝) 채우기
    loader_name = getattr(ohlc_loader, "__name__", "ohlc_loader")
    summary = _add_price_data(summary, wd, ohlc_loader, loader_name)
    summary = _fill_price_data(summary)

    # 3) 이진 H0/H1 + P0/P1 생성
    bin_df = _generate_binary_hike_info(summary, num_upcoming)

    # 4) watch_date 기준 현재 타깃금리 구간
    if rate_cols and watch_rate_range is None:
        try:
            if wd >= datetime(2008, 12, 16):
                ll = pdr.DataReader("DFEDTARL", "fred", start=wd, end=wd).iloc[0, 0]
                ul = pdr.DataReader("DFEDTARU", "fred", start=wd, end=wd).iloc[0, 0]
                watch_rate_range = (ll, ul)
            else:
                ll = ul = pdr.DataReader("DFEDTAR", "fred", start=wd, end=wd).iloc[0, 0]
                watch_rate_range = (ll, ul)
        except Exception as e:
            raise ValueError(
                "FRED에서 타깃금리 범위를 가져오지 못했습니다. "
                "watch_rate_range=(ll, ul)을 직접 지정해 주세요."
            ) from e

    # 5) 회의별 누적 hike size / prob 계산 (convolution)
    def _extract(group: pd.DataFrame) -> dict:
        return {
            "hike_size": np.array(group[["H0", "H1"]].values[0]),
            "hike_prob": np.array(group[["P0", "P1"]].values[0]),
        }

    grouped = bin_df.groupby("Meeting").apply(_extract)

    # 첫 회의
    first_meeting = grouped.index[0]
    ms = grouped.iloc[0]["hike_size"]
    mp = grouped.iloc[0]["hike_prob"]

    data = {"FOMCDate": first_meeting}
    for size, prob in zip(ms, mp):
        data[size] = [prob]

    fedwatch_df = pd.DataFrame(data).set_index("FOMCDate")

    lead_size, lead_prob = ms, mp

    # 이후 회의들
    for i in range(1, len(grouped)):
        meet_date = grouped.index[i]
        lag_size = grouped.iloc[i]["hike_size"]
        lag_prob = grouped.iloc[i]["hike_prob"]

        ms, mp = _calc_cum_info(lead_size, lag_size, lead_prob, lag_prob)

        data = {"FOMCDate": meet_date}
        for size, prob in zip(ms, mp):
            data[size] = [prob]

        fedwatch_df = pd.concat(
            [fedwatch_df, pd.DataFrame(data).set_index("FOMCDate")]
        ).fillna(0.0)

        lead_size, lead_prob = ms, mp

    fedwatch_df.sort_index(axis=1, inplace=True)

    # watch_date MultiIndex 추가
    fedwatch_df["WatchDate"] = wd.strftime("%Y-%m-%d")
    fedwatch_df.reset_index(inplace=True)
    fedwatch_df.set_index(["WatchDate", "FOMCDate"], inplace=True)

    # 6) 컬럼명을 실제 타깃금리 구간으로 변환 (선택)
    if rate_cols:
        ll, ul = watch_rate_range
        diff = ul - ll
        new_cols = []
        for c in fedwatch_df.columns:
            col_bp = float(c) / 100.0  # 25 → 0.25bp
            if diff == 0:
                new_name = f"{ll + col_bp:.2f}"
            else:
                new_name = f"{ll + col_bp:.2f}-{ul + col_bp:.2f}"
            new_cols.append(new_name)
        fedwatch_df.columns = new_cols

    return fedwatch_df

In [4]:
import time

from tvDatafeed import TvDatafeed, Interval
from functools import lru_cache

tv = TvDatafeed()

FOMC_DATES = [
    "2001-01-03", "2001-01-31", "2001-03-20", "2001-04-18", "2001-05-15",
    "2001-06-27", "2001-08-21", "2001-09-17", "2001-10-02", "2001-11-06",
    "2001-12-11",
    "2002-11-06",
    "2003-06-25",
    "2004-06-30", "2004-08-10", "2004-09-21", "2004-11-10", "2004-12-14",
    "2005-02-02", "2005-03-22", "2005-05-03", "2005-06-30", "2005-08-09",
    "2005-09-20", "2005-11-01", "2005-12-13",
    "2006-01-31", "2006-03-28", "2006-05-10", "2006-06-29",
    "2007-09-18", "2007-10-31", "2007-12-11",
    "2008-01-22", "2008-01-30", "2008-03-18", "2008-04-30", "2008-06-25",
    "2008-08-05", "2008-09-16", "2008-10-08", "2008-10-29", "2008-12-16",
    "2009-01-28", "2009-03-18", "2009-04-29", "2009-06-24", "2009-08-12",
    "2009-09-23", "2009-11-04", "2009-12-16",
    "2010-01-27", "2010-03-16", "2010-04-28", "2010-06-23", "2010-08-10",
    "2010-09-21", "2010-11-03", "2010-12-14",
    "2011-01-26", "2011-03-15", "2011-04-27", "2011-06-22", "2011-08-09",
    "2011-09-21", "2011-11-02", "2011-12-13",
    "2012-01-25", "2012-03-13", "2012-04-25", "2012-06-20", "2012-08-01",
    "2012-09-13", "2012-10-24", "2012-12-12",
    "2013-01-30", "2013-03-20", "2013-05-01", "2013-06-19", "2013-07-31",
    "2013-09-18", "2013-10-30", "2013-12-18",
    "2014-01-29", "2014-03-19", "2014-04-30", "2014-06-18", "2014-07-30",
    "2014-09-17", "2014-10-29", "2014-12-17",
    "2015-01-28", "2015-03-18", "2015-04-29", "2015-06-17", "2015-07-29",
    "2015-09-17", "2015-10-28", "2015-12-16",
    "2016-01-27", "2016-03-16", "2016-04-27", "2016-06-15", "2016-07-27",
    "2016-09-21", "2016-11-02", "2016-12-14",
    "2017-02-01", "2017-03-15", "2017-05-03", "2017-06-14", "2017-07-26",
    "2017-09-20", "2017-11-01", "2017-12-13",
    "2018-01-31", "2018-03-21", "2018-05-02", "2018-06-13", "2018-08-01",
    "2018-09-26", "2018-11-08", "2018-12-19",
    "2019-01-30", "2019-03-20", "2019-05-01", "2019-06-19", "2019-07-31",
    "2019-09-18", "2019-10-30", "2019-12-11",
    "2020-01-29", "2020-03-03", "2020-03-15", "2020-04-29", "2020-06-10",
    "2020-07-29", "2020-09-16", "2020-11-05", "2020-12-16",
    "2021-01-27", "2021-03-17", "2021-04-28", "2021-06-16", "2021-07-28",
    "2021-09-22", "2021-11-03", "2021-12-15",
    "2022-01-26", "2022-03-16", "2022-05-04", "2022-06-15", "2022-07-27",
    "2022-09-21", "2022-11-02", "2022-12-14",
    "2023-02-01", "2023-03-22", "2023-05-03", "2023-06-14", "2023-07-26",
    "2023-09-20", "2023-11-01", "2023-12-13",
    "2024-01-31", "2024-03-20", "2024-05-01", "2024-06-12", "2024-07-31",
    "2024-09-18", "2024-11-07", "2024-12-18",
    "2025-01-29", "2025-03-19", "2025-05-07", "2025-06-18", "2025-07-30",
    "2025-09-17", "2025-10-29", "2025-12-10",
    "2026-01-28",'2026-03-18','2026-04-29','2026-06-17',
    '2026-07-29','2026-09-16','2026-10-28','2026-12-09'
]

def make_cached_loader(base_loader):
    """
    base_loader(symbol: str) -> DataFrame  형태의 함수를
    심볼별 1번만 호출하도록 캐시를 씌운 래퍼 생성.
    """
    @lru_cache(None)
    def _cached(symbol: str):
        df = base_loader(symbol)
        # 혹시 모를 사이드이펙트 방지를 위해 복사본 반환
        return df.copy()
    return _cached

def load_zq_from_tv(symbol: str,
                    max_retries: int = 10,
                    sleep_sec: float = 1.0) -> pd.DataFrame:
    """
    TradingView(tvDatafeed)에서 ZQ 선물 데이터를 가져올 때,
    실패 시 최대 max_retries 번까지 재시도하는 함수.

    반환: Date 인덱스, 'Close' 컬럼 포함 DataFrame
    """
    last_exc = None

    for attempt in range(1, max_retries + 1):
        try:
            zq = tv.get_hist(
                symbol=symbol,
                exchange="CBOT",
                interval=Interval.in_daily,
                n_bars=2000,
            )

            # tvDatafeed가 None 또는 빈 DataFrame을 줄 가능성 방어
            if zq is None or zq.empty:
                raise ValueError(f"No data returned from TradingView for {symbol}")

            df = zq.copy()
            df = df.rename_axis("Date").reset_index()
            df.rename(columns={"close": "Close"}, inplace=True)

            # 여기까지 오면 성공
            return df

        except Exception as e:
            last_exc = e
            # 필요하면 로그 출력
            print(f"[{symbol}] TV fetch failed on attempt {attempt}/{max_retries}: {e}")
            if attempt < max_retries:
                time.sleep(sleep_sec)

    # 모든 재시도 실패 시 예외 발생
    raise RuntimeError(
        f"Failed to load TradingView data for {symbol} after {max_retries} attempts"
    ) from last_exc

def extract_nearest_meeting_probs(fedwatch_df: pd.DataFrame) -> pd.Series:
    """
    compute_fedwatch_prob_table 결과에서
    - 해당 WatchDate 기준 가장 가까운 FOMCDate 하나만 선택
    - 그 회의에 대한 금리 구간별 확률(row)을 Series로 반환

    반환: index = '3.00-3.25', '3.25-3.50', ... / name = WatchDate
    """
    df = fedwatch_df.reset_index()  # WatchDate, FOMCDate 컬럼 복원

    # 가장 가까운 (가장 이른) FOMCDate 선택
    nearest_date = df["FOMCDate"].min()
    row = df[df["FOMCDate"] == nearest_date].iloc[0]

    # 금리 구간 컬럼만 추출 (예: '5.25-5.50')
    rate_cols = [c for c in df.columns if isinstance(c, str) and "-" in c]

    s = row[rate_cols]
    # WatchDate를 index name으로 쓰기 위해 name 세팅
    s.name = pd.to_datetime(row["WatchDate"])
    return s

you are using nologin method, data you access may be limited


In [5]:
from tqdm import tqdm

def build_fedwatch_nearest_ts(
    start_date: str | pd.Timestamp,
    end_date: str | pd.Timestamp,
    fomc_dates: list,
    ohlc_loader,                 # 예: cached_loader
    num_upcoming: int = 4,
    rate_cols: bool = True,
    watch_rate_range=None,
    freq: str = "B",             # Business day 기준
) -> pd.DataFrame:
    """
    start_date ~ end_date 구간의 watch_date에 대해,
    - FedWatch 확률 테이블을 계산하고
    - 각 날짜마다 '가장 가까운 FOMC'의 금리 구간 확률만 추출
    - 이를 이어붙여 시계열 DataFrame 생성

    반환:
        index: WatchDate (DatetimeIndex)
        columns: '3.00-3.25', '3.25-3.50', ... 금리구간
    """
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # 심볼 다운로드 최소화를 위해 loader에 캐시 적용 (필요시 한 번만)
    cached_loader = make_cached_loader(ohlc_loader)

    watch_dates = pd.date_range(start_date, end_date, freq=freq)

    rows = []

    for wd in tqdm(watch_dates):
        try:
            fw = compute_fedwatch_prob_table(
                watch_date=wd,
                fomc_dates=fomc_dates,
                num_upcoming=num_upcoming,
                ohlc_loader=cached_loader,
                rate_cols=rate_cols,
                watch_rate_range=watch_rate_range,
            )
        except ValueError as e:
            # 예: 해당 날짜 기준으로 유효한 회의가 num_upcoming보다 적을 때 등
            # 이런 경우는 그냥 건너뜀
            continue

        s = extract_nearest_meeting_probs(fw)
        rows.append(s)

    if not rows:
        return pd.DataFrame()

    ts_df = pd.DataFrame(rows)
    ts_df.index.name = "WatchDate"
    ts_df.sort_index(inplace=True)
    return ts_df

In [6]:
cached_loader = make_cached_loader(load_zq_from_tv)

fed_ts = build_fedwatch_nearest_ts(
    start_date="2004-01-01",
    end_date="2026-12-31",
    fomc_dates=FOMC_DATES,
    ohlc_loader=cached_loader,
    num_upcoming=4,
    rate_cols=True,
    # watch_rate_range=(5.25, 5.50),
)

ERROR! Session/line number was not unique in database. History logging moved to new session 2438


100%|██████████| 6001/6001 [18:08<00:00,  5.52it/s]  


In [7]:
fed_ts

,-5.25,-5.00,-4.75,-4.50,-4.25,0.00-0.25,0.25-0.50,0.50-0.75,0.75-1.00,1.00-1.25,...,3.75-4.00,4.00-4.25,4.25-4.50,4.50-4.75,4.75-5.00,5.00-5.25,5.25-5.50,5.50-5.75,5.75-6.00,6.00-6.25
WatchDate,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.111481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.128519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.148519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
fed_ts.to_excel('fed_prob.xlsx')

In [21]:
fed_ts[fed_ts.fillna(0).diff() > 0.3].iloc[:,5:]['0.00-0.25'].dropna()

WatchDate
2008-12-16    0.977037
2010-06-23    0.930000
2011-01-13    0.987600
2011-01-26    0.940000
2011-03-28    0.976923
2011-03-30    0.976923
2012-03-28    1.000000
2012-04-03    0.987500
2012-04-10    0.987500
2012-04-17    0.987500
2012-05-11    0.990000
2012-05-24    0.990000
2012-06-21    0.980000
2012-07-13    0.980000
2012-08-24    0.982609
2013-02-18    0.990000
2013-05-07    0.966667
2013-05-21    0.966667
2013-05-27    0.966667
2013-12-03    0.997857
2014-04-24    0.989655
2014-10-02    0.977857
2015-03-25    0.978571
2015-04-07    0.989286
2020-03-16    0.871429
2020-03-18    0.978571
2020-03-24    0.978571
2020-03-27    0.978571
2020-05-13    0.990000
2020-05-19    0.947857
2020-05-26    0.990000
2020-06-02    0.977857
2021-01-05    0.976154
2021-03-31    0.988889
2021-06-21    0.977037
2021-06-23    0.977037
2021-06-30    0.988519
Name: 0.00-0.25, dtype: float64